In [1]:
#dependencies
!python -m pip install numpy-stl

In [2]:
#imports
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import numpy as np
from scipy.spatial import Delaunay
from shutil import copyfile

from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot
%matplotlib notebook

dataset_random_seed = 0
dataset_size = 50

In [3]:
#generate 2D rectangles - dataset 1
np.random.seed(dataset_random_seed)

def random_rectangle(min_scale, max_scale):
    scale = np.random.rand()*(max_scale-min_scale)+min_scale
    width, height = np.random.rand(2)*scale
    return np.round([0, 0, 0, width, height, width, height, 0],5)

rectangles = list()
for i in range(0,dataset_size):
    rectangles.append(random_rectangle(1,3))
    
with open('dataset_1.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    for i in range(0,dataset_size):
        csv_writer.writerow(rectangles[i])

In [4]:
#generate 2D convex polygons - dataset 2
np.random.seed(dataset_random_seed)

def random_polygon_convex(min_scale, max_scale, min_vertices, max_vertices):
    n_vertices = np.random.randint(min_vertices,max_vertices)
    scale = np.random.rand()*(max_scale-min_scale)+min_scale
    points = np.random.rand(n_vertices, 2)*scale
    hull = ConvexHull(points)

    vertices = points[hull.vertices,:]
    return np.round(vertices,5)

convex_polygons = list()
for i in range(0,dataset_size):
    convex_polygons.append(random_polygon_convex(1,3,3,12))
    
with open('dataset_2.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    for i in range(0,dataset_size):
        csv_writer.writerow(convex_polygons[i].flatten())

In [5]:
#generate 2D non-convex polygons - dataset 3
np.random.seed(dataset_random_seed)

def ccw(Ax,Bx,Cx,Ay,By,Cy):
    return (Cy-Ay) * (Bx-Ax) > (By-Ay) * (Cx-Ax)

# Return true if line segments AB and CD intersect
def intersect(Ax,Bx,Cx,Dx, Ay,By,Cy,Dy):
    return ccw(Ax,Cx,Dx,Ay,Cy,Dy) != ccw(Bx,Cx,Dx,By,Cy,Dy) and ccw(Ax,Bx,Cx,Ay,By,Cy) != ccw(Ax,Bx,Dx,Ay,By,Dy)

def count_intersections(vertices):
    n_vertices = vertices.shape[0]
    M=np.zeros((n_vertices,n_vertices))
    vertices = np.vstack((vertices,vertices[0,:]))
    for x in range(0,n_vertices):
        for y in range(x+2,n_vertices):
            M[x,y] = intersect(vertices[x,0],vertices[x+1,0],vertices[y,0],vertices[y+1,0],vertices[x,1],vertices[x+1,1],vertices[y,1],vertices[y+1,1])
    vertices = vertices[0:-1,:]
    return np.sum(M)

def generate_concave_polygon(n_vertices=8):#because of the random part, this is impractical for n_vertices>10
    vertices = np.random.random((n_vertices,2))
    while count_intersections(vertices)>0:
        np.random.shuffle(vertices)
    return vertices

def random_polygon_concave(min_scale, max_scale, min_vertices, max_vertices):
    n_vertices = np.random.randint(min_vertices,max_vertices)
    scale = np.random.rand()*(max_scale-min_scale)+min_scale

    vertices = generate_concave_polygon(n_vertices)*scale
    return np.round(vertices,5)

concave_polygons = list()
for i in range(0,dataset_size):
    concave_polygons.append(random_polygon_concave(1,3,5,10))
    
with open('dataset_3.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    for i in range(0,dataset_size):
        csv_writer.writerow(concave_polygons[i].flatten())

In [6]:
#dataset 4 is the union of datasets 1, 2, and 3
filenames = ['dataset_1.csv', 'dataset_2.csv', 'dataset_3.csv']
with open('dataset_4.csv', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())

In [7]:
#visualise 2D datasets
vertices = rectangles[49].reshape(4,2)
disp_vertices = np.vstack((vertices,vertices[0,:]))
plt.plot(disp_vertices[:,0],disp_vertices[:,1])
plt.show()
vertices = convex_polygons[49]
disp_vertices = np.vstack((vertices,vertices[0,:]))
plt.plot(disp_vertices[:,0],disp_vertices[:,1])
plt.show()
vertices = concave_polygons[49]
disp_vertices = np.vstack((vertices,vertices[0,:]))
plt.plot(disp_vertices[:,0],disp_vertices[:,1])
plt.show()

<IPython.core.display.Javascript object>

In [8]:
#from https://stackoverflow.com/questions/24733185/volume-of-convex-hull-with-qhull-from-scipy
def tetrahedron_volume(a, b, c, d):
    return np.abs(np.einsum('ij,ij->i', a-d, np.cross(b-d, c-d))) / 6

def convex_hull_volume_bis(pts):
    ch = ConvexHull(pts)

    simplices = np.column_stack((np.repeat(ch.vertices[0], ch.nsimplex),
                                 ch.simplices))
    tets = ch.points[simplices]
    return np.sum(tetrahedron_volume(tets[:, 0], tets[:, 1],
                                     tets[:, 2], tets[:, 3]))

def convex_hull_of_mesh(mesh):
    points = np.concatenate([mesh.v0,mesh.v1,mesh.v2])
    return convex_hull_volume_bis(points)

In [9]:
#generate 3D cuboids - dataset 5
np.random.seed(dataset_random_seed)
cuboid_volumes = np.zeros(dataset_size)

# Define the 12 triangles composing the cuboid (static)
faces = np.array([\
    [0,3,1],
    [1,3,2],
    [0,4,7],
    [0,7,3],
    [4,5,6],
    [4,6,7],
    [5,1,2],
    [5,2,6],
    [2,3,6],
    [3,7,6],
    [0,1,5],
    [0,5,4]])

def random_cuboid(min_scale, max_scale):
    scale = np.random.rand()*(max_scale-min_scale)+min_scale
    width, height, depth = np.round(np.random.rand(3)*scale,5)
    # Define the 8 vertices of the cuboid (dynamic)
    vertices = np.array([\
        [-width, -height, -depth],
        [+width, -height, -depth],
        [+width, +height, -depth],
        [-width, +height, -depth],
        [-width, -height, +depth],
        [+width, -height, +depth],
        [+width, +height, +depth],
        [-width, +height, +depth]])*scale/2
    
    # Create the mesh
    cuboid = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            cuboid.vectors[i][j] = vertices[f[j],:]
    return cuboid
        
for i in range(0,dataset_size):
    cuboid = random_cuboid(1,3)
    points = np.concatenate([cuboid.v0,cuboid.v1,cuboid.v2])
    cuboid_volumes[i] = convex_hull_volume_bis(points)
    cuboid.save('dataset_5/'+str(i)+'.stl')

In [10]:
#generate 3D convex polytopes - dataset 6
np.random.seed(dataset_random_seed)
convex_volumes = np.zeros(dataset_size)

def generate_convex_polyhedra(min_scale, max_scale, min_vertices, max_vertices):
    scale = np.random.rand()*(max_scale-min_scale)+min_scale
    n_vertices = np.random.randint(min_vertices,max_vertices)

    points = np.round(np.random.rand(n_vertices, 3),5)*scale
    hull = ConvexHull(points)
    return points, hull.simplices

for n in range(0,dataset_size):
    polygon_points, polygon_triangles = generate_convex_polyhedra(1,3,4,15)

    convex_polytope = mesh.Mesh(np.zeros(polygon_triangles.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(polygon_triangles):
        for j in range(3):
            convex_polytope.vectors[i][j] = polygon_points[f[j],:]
            
    convex_volumes[i] = convex_hull_volume_bis(polygon_points)
    convex_polytope.save('dataset_6/'+str(n)+'.stl')

In [11]:
#generate 3D non-convex polytopes - dataset 7
np.random.seed(dataset_random_seed)
concave_volumes = np.zeros(dataset_size)

def generate_concave_polyhedra(min_scale, max_scale, min_vertices, max_vertices):
    #first, generate a convex polyhedron
    polygon_points, simplices = generate_convex_polyhedra(1,1,min_vertices,max_vertices)

    #random point inside polytope
    random_point = np.random.rand(3)
    #Delaunay(polygon_points).find_simplex(random_point) >= 0 is True if point lies within poly
    while ~(Delaunay(polygon_points).find_simplex(random_point) >= 0):  # True if point lies within poly
        random_point = np.random.rand(3)

    #select random face
    random_simplex = np.random.randint(0,len(simplices))

    #remove this face, and replace it with three faces between its edges and the random point
    polygon_points = np.concatenate((polygon_points,[random_point]))
    new_point = len(polygon_points)-1

    removed_simplex = simplices[random_simplex]
    simplices = simplices[np.arange(len(simplices))!=random_simplex] #remove the face

    #add three faces
    simplices = np.concatenate((simplices,
          [[removed_simplex[0], removed_simplex[1], new_point],
           [new_point, removed_simplex[1], removed_simplex[2]],
           [removed_simplex[0], new_point, removed_simplex[2]]]))

    scale = np.random.rand()*(max_scale-min_scale)+min_scale
    return np.round(polygon_points*scale,5), simplices


for n in range(0,dataset_size):
    polygon_points, simplices = generate_concave_polyhedra(1,3,4,15)

    concave_polytope = mesh.Mesh(np.zeros(simplices.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(simplices):
        for j in range(3):
            concave_polytope.vectors[i][j] = polygon_points[f[j],:]

    concave_volumes[i] = convex_hull_volume_bis(polygon_points)#actually the volume of the convex hull

    concave_polytope.save('dataset_7/'+str(n)+'.stl')

In [12]:
#dataset 8 is the union of datasets 5, 6, and 7
for d in range(5,8):#dataset number
    for n in range(0,dataset_size):#polytope number
        copyfile('dataset_'+str(d)+'/'+str(n)+'.stl', 'dataset_8/'+str(n+(d-5)*50)+'.stl')

In [13]:
#visualise 3D datasets
figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

axes.add_collection3d(mplot3d.art3d.Poly3DCollection(cuboid.vectors))

scale = cuboid.points.flatten()
axes.auto_scale_xyz(scale, scale, scale)

pyplot.show()

figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

axes.add_collection3d(mplot3d.art3d.Poly3DCollection(convex_polytope.vectors))

scale = convex_polytope.points.flatten()
axes.auto_scale_xyz(scale, scale, scale)

pyplot.show()

figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

axes.add_collection3d(mplot3d.art3d.Poly3DCollection(concave_polytope.vectors))

scale = concave_polytope.points.flatten()
axes.auto_scale_xyz(scale, scale, scale)

pyplot.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
#run Blender rigid body physics simulation on each dataset, and export as dataset_6_after_physics_packing.stl

In [15]:
# Create a new plot
figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

# Load the STL files and add the vectors to the plot
your_mesh = mesh.Mesh.from_file('dataset_6_after_physics_packing.stl')
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

# Auto scale to the mesh size
scale = your_mesh.points.flatten()
axes.auto_scale_xyz(scale, scale, scale)

# Show the plot to the screen
pyplot.show()

<IPython.core.display.Javascript object>

In [16]:
#convert dataset_6_after_physics_packing.stl into list of objects
objects = []
#keep adding to the list of objects until all vertices are taken
while objects==[] or len(np.concatenate(objects).ravel())<len(your_mesh.v0):
    if objects==[]:
        #initialization (first itteration)
        this_object_triangles = [0]
    else:
        #find first triangle not among objects
        this_object_triangles = [np.setdiff1d(range(len(your_mesh.v0)), np.concatenate(objects).ravel())[0]]

    cur_num_triangles = 0
    #find vertices that match any of the vertices that are in this_object_triangles (faces)
    while len(this_object_triangles)>cur_num_triangles:
        cur_num_triangles = len(this_object_triangles)
        #find connected vertices
        for known_faces in this_object_triangles:
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v0==your_mesh.v0[known_faces],1))[0])
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v1==your_mesh.v0[known_faces],1))[0])
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v2==your_mesh.v0[known_faces],1))[0])
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v0==your_mesh.v1[known_faces],1))[0])
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v1==your_mesh.v1[known_faces],1))[0])
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v2==your_mesh.v1[known_faces],1))[0])
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v0==your_mesh.v2[known_faces],1))[0])
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v1==your_mesh.v2[known_faces],1))[0])
            this_object_triangles = np.union1d(this_object_triangles, np.where(np.all(your_mesh.v2==your_mesh.v2[known_faces],1))[0])
    objects.append(this_object_triangles)
print(len(objects))
#there should be 50 objects in dataset 6

50


In [29]:
polytope = convex_polytope
#check that these are the same objects as before
#use 1)number of simplices and 2)volume of convex hull as polytope identifier
objects_with_same_number_of_sides = np.where([len(o)==len(polytope.v0) for o in objects])[0] #actually number of triangles

ref_volume = convex_hull_of_mesh(polytope)
print(ref_volume)#volume
for candidate_objects in objects_with_same_number_of_sides:
    this_candidate = mesh.Mesh(your_mesh.data[objects[candidate_objects]])
    print(np.abs(ref_volume-convex_hull_of_mesh(this_candidate))<0.0001) #volume diff less than epsilon

1.8813904891856494
False
False
False
False
False
False
False
True


In [ ]:
#find object intersectons when convex (untested code)
from scipy.spatial import ConvexHull, intersection

def intersect_polytopes(polytope1, polytope2):
    points1, simplices1 = polytope1
    points2, simplices2 = polytope2

    # Create ConvexHull objects for both polytopes
    hull1 = ConvexHull(points1, simplices1)
    hull2 = ConvexHull(points2, simplices2)

    # Find the intersection of the two ConvexHull objects
    intersect = intersection(hull1, hull2)

    # Return the intersection as a list of 3D points
    return intersect.points.tolist()

    
#find object intersections when non-convex (untested code)
from scipy.spatial import ConvexHull, Delaunay, convex_hull_intersection

def intersect_polytopes(polytope1, polytope2):
    points1, simplices1 = polytope1
    points2, simplices2 = polytope2

    # Split each polytope into its convex parts
    tri1 = Delaunay(points1)
    convex_parts1 = [ConvexHull(points1[tri1.vertices[f]]) for f in tri1.simplices]
    tri2 = Delaunay(points2)
    convex_parts2 = [ConvexHull(points2[tri2.vertices[f]]) for f in tri2.simplices]

    # Find the intersection of each pair of convex parts
    intersections = []
    for convex_part1 in convex_parts1:
        for convex_part2 in convex_parts2:
            intersection = convex_hull_intersection(convex_part1, convex_part2)
            if intersection.points.shape[0] > 0:
                intersections.append(intersection)

    # Join the resulting convex intersections to create the intersection of the non-convex polytopes
    if len(intersections) == 0:
        # Return an empty intersection if no intersections were found
        return np.empty((0, 3)), np.empty((0, 3))
    else:
        points = np.vstack([intersection.points for intersection in intersections])
        tri = Delaunay(points)
        return points, tri.simplices